In [ ]:
!pip install --upgrade pip
!pip install pyyaml
!pip install ultralytics
!apt-get update && apt-get install libgl1-mesa-glx -y

# pip installs to get YOLO working!

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# mounting google drive so we can pull in the data and directory

In [ ]:
import pandas as pd
import os
import shutil
import yaml
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

# imports:
# shututil used for file operations
# yaml for YOLO config file
# sklearn to train test split
# ultralytics to run YOLO

In [ ]:
images_load = '/content/drive/My Drive/D144/people_drone_detection_data/images/images' # load in images
annotations_load = '/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format' # load in annotations

In [ ]:
images = []
annotations = []
# initialization


# iterate over each file in images and annotation as a pair to create a dataframe with both aligned
for img_file in os.listdir(images_load):
    if img_file.endswith(".jpg"):
        image_name = os.path.splitext(img_file)[0]
        annotation_name = image_name + ".txt"
        if os.path.exists(os.path.join(annotations_load, annotation_name)):
            images.append(os.path.join(images_load, img_file))
            annotations.append(os.path.join(annotations_load, annotation_name))



df = pd.DataFrame({"images": images, "annotations": annotations}) # creating the dataframe of images and annotations

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train_df, test_size=0.4, random_state=42)
# train test splitting into train and test, and further splitting train into train and val to have train, val, test

In [ ]:
# Create directories for annotation image pairings for train, test, val
os.makedirs(os.path.join("/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format", "train", "images"), exist_ok = True)
os.makedirs(os.path.join("/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format", "train", "annotations"), exist_ok = True)
os.makedirs(os.path.join("/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format", "val", "images"), exist_ok = True)
os.makedirs(os.path.join("/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format", "val", "annotations"), exist_ok = True)
os.makedirs(os.path.join("/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format", "test", "images"), exist_ok = True)
os.makedirs(os.path.join("/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format", "test", "annotations"), exist_ok = True)

In [ ]:
# training set image/annotation allocation:

images_dir_train = "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/train/images"
annotations_dir_train = "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/train/annotations"

def update_filepath_train(df):
  # extract from df
    images = df["images"]
    annotations = df["annotations"]

  # get filenames
    image_dest_end =  os.path.basename(images)
    annotations_dest_end = os.path.basename(annotations)

  # set destination paths
    image_dest = os.path.join(images_dir_train, image_dest_end)
    annotations_dest = os.path.join(annotations_dir_train, annotations_dest_end)

  # move to train
    shutil.move(images, image_dest)
    shutil.move(annotations, annotations_dest)

print("start train")
train2 = train.apply(update_filepath_train, axis = 1) # for each row in train
print("done train")

# validation set image/annotation allocation:

images_dir_val = "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/val/images"
annotations_dir_val = "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/val/annotations"

def update_filepath_val(df):
  # extract from df
    images = df["images"]
    annotations = df["annotations"]

  # get filenames
    image_dest_end =  os.path.basename(images)
    annotations_dest_end = os.path.basename(annotations)

  # set destination paths
    image_dest = os.path.join(images_dir_val, image_dest_end)
    annotations_dest = os.path.join(annotations_dir_val, annotations_dest_end)

  # move to val
    shutil.move(images, image_dest)
    shutil.move(annotations, annotations_dest)

print("start val")
val2 = val.apply(update_filepath_val, axis = 1) # for each row in val
print("done val")

# test set image/annotation allocation:

images_dir_test = "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/test/images"
annotations_dir_test = "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/test/annotations"

def update_filepath_test(df):
  # extract from df
    images = df["images"]
    annotations = df["annotations"]

  # get filenames
    image_dest_end =  os.path.basename(images)
    annotations_dest_end = os.path.basename(annotations)

  # set destination paths
    image_dest = os.path.join(images_dir_test, image_dest_end)
    annotations_dest = os.path.join(annotations_dir_test, annotations_dest_end)

  # move to val
    shutil.move(images, image_dest)
    shutil.move(annotations, annotations_dest)

    return pd.Series([image_dest, annotations_dest], index = ["images", "annotations"])


print("start test")
test2 = test_df.apply(update_filepath_test, axis = 1) # for each row in test
print("end test")
train2, val2, test2

In [ ]:
config = {
    "nc" : 1, # only human class (y/n binary classification)
    "names": ["human"],
    "path": "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format",
    "train": "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/train",
    "val": "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/val",
    "test": "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/test"
}

# setup/configuration file - yaml used for YOLO model

with open("config.yaml", "w") as f:
    yaml.dump(config, f)

with open("config.yaml", "r") as f:
    print(f.read())

config # confirming nit was correctly written

In [ ]:
path = "/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/config.yaml"
print(f"Config file created: {os.path.exists(path)}")
# confirming config file exists in our YOLO-format directory

In [ ]:
model = YOLO("yolov8n.pt") # loading pretrained model

model.train(
        # defining where we will house the model-related components
        project="/content/drive/My Drive/D144/people_drone_detection_data",
        name="yolov8n",
        deterministic=True,
        seed=42,
        data="/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/config.yaml",
        save=True,
        save_period=1,
        pretrained=True,
        imgsz=1280,
        epochs=5,
        batch=-1,
        workers=10,
        val=True,
        lr0=0.01,
        patience=3,
        optimizer = 'SGD',
) # parameters for YOLO

In [ ]:
metrics = model.val() # let's see how we did!
print(metrics)

In [ ]:
results = model.val(save=True, save_dir="/content/drive/My Drive/D144/people_drone_detection_data") # save the metrics!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# taking our metrics csv to plot precision recall and mAP
metrics = pd.read_csv('/content/drive/My Drive/D144/people_drone_detection_data/results.csv')

# plots over epochs
plt.figure(figsize=(14, 6))

# precision
plt.subplot(1, 3, 1)
plt.plot(metrics['epoch'], metrics['metrics/precision(B)'], label='Precision', color='b')
plt.title('Precision over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.grid(True)

# recall
plt.subplot(1, 3, 2)
plt.plot(metrics['epoch'], metrics['metrics/recall(B)'], label='Recall', color='r')
plt.title('Recall over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.grid(True)

# mAP
plt.subplot(1, 3, 3)
plt.plot(metrics['epoch'], metrics['metrics/mAP50(B)'], label='mAP', color='g')
plt.title('mAP50 over Epochs')
plt.xlabel('Epochs')
plt.ylabel('mAP')
plt.grid(True)

In [ ]:
# plotting the losses over epochs
plt.figure(figsize=(14, 6))

# box loss - training
plt.subplot(1, 2, 1)
plt.plot(metrics['epoch'], metrics['train/box_loss'], label='Train Box Loss', color='b')
plt.title('Train Box Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Box Loss')
plt.grid(True)

# box loss - validation
plt.subplot(1, 2, 2)
plt.plot(metrics['epoch'], metrics['val/box_loss'], label='Validation Box Loss', color='r')
plt.title('Validation Box Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Box Loss')
plt.grid(True)

In [ ]:
model = YOLO('/content/drive/My Drive/D144/people_drone_detection_data/yolov8n3/weights/best.pt')  # reloading model after crash using best weight

In [ ]:
test_results = model.predict(source="/content/drive/My Drive/D144/people_drone_detection_data/annotation/annotation/YOLO-format/test/images", save=True, save_dir="/content/drive/My Drive/D144/people_drone_detection_data") # predicting on test set!

In [ ]:
shutil.move('/content/runs/detect/predict', '/content/drive/My Drive/D144/people_drone_detection_data') # saving our predictions!